# Milvus Vector Store

In this notebook we are going to show a quick demo of using the MilvusVectorStore. 

In [1]:
import logging
import sys

# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.vector_stores import MilvusVectorStore
from IPython.display import Markdown, display
import textwrap

/Users/filiphaltmayer/miniconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup OpenAI
Lets first begin by adding the openai api key. This will allow us to access openai for embeddings and to use chatgpt.

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"

### Generate our data
With our LLM set, lets start using the Milvus Index. As a first example, lets generate a document from the file found in the `paul_graham_essay/data` folder. In this folder there is a single essay from Paul Graham titled `What I Worked On`. To generate the documents we will use the SimpleDirectoryReader.

In [3]:
# load documents
documents = SimpleDirectoryReader('../data/paul_graham/').load_data()
print('Document ID:', documents[0].doc_id, 'Document Hash:', documents[0].doc_hash)

Document ID: 05b6691b-d567-43a2-94e1-e9ca81cd4624 Document Hash: 77ae91ab542f3abb308c4d7c77c9bc4c9ad0ccd63144802b7cbe7e1bb3a4094e


### Create an index across the data
Now that we have a document, we can can create an index and insert the document. For the index we will use a GPTMilvusIndex. GPTMilvusIndex takes in a few arguments:

- collection_name (str, optional): The name of the collection where data will be stored. Defaults to "llamalection".
- index_params (dict, optional): The index parameters for Milvus, if none are provided an HNSW index will be used. Defaults to None.
- search_params (dict, optional): The search parameters for a Milvus query. If none are provided, default params will be generated. Defaults to None.
- dim (int, optional): The dimension of the embeddings. If it is not provided, collection creation will be done on first insert. Defaults to None.
- host (str, optional): The host address of Milvus. Defaults to "localhost".
- port (int, optional): The port of Milvus. Defaults to 19530.
- user (str, optional): The username for RBAC. Defaults to "".
- password (str, optional): The password for RBAC. Defaults to "".
- use_secure (bool, optional): Use https. Defaults to False.
- overwrite (bool, optional): Whether to overwrite existing collection with same name. Defaults to False.


In [4]:
# Create an index over the documnts
from llama_index.storage.storage_context import StorageContext


vector_store = MilvusVectorStore(overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context)

### Query the data
Now that we have our document stored in the index, we can ask questions against the index. The index will use the data stored in itself as the knowledge base for chatgpt.

In [5]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author learn?")
print(textwrap.fill(str(response), 100))

 The author learned that the AI programs of the time were not capable of understanding natural
language, and that the field of AI was a hoax. He also learned that he could make art, and that he
could pass the entrance exam for the Accademia di Belli Arti in Florence. He also learned Lisp
hacking and wrote his dissertation on applications of continuations.


In [6]:
response = query_engine.query("What was a hard moment for the author?")
print(textwrap.fill(str(response), 100))

 A hard moment for the author was when he realized that the AI programs of the time were a hoax and
that there was an unbridgeable gap between what they could do and actually understanding natural
language.


This next test shows that overwriting removes the previous data.

In [7]:
vector_store = MilvusVectorStore(overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents([Document("The number that is being searched for is ten.")], storage_context)
query_engine = index.as_query_engine()
res = query_engine.query("Who is the author?")
print("Res:", res)

Res: 
The author is unknown.


The next test shows adding additional data to an already existing  index.

In [8]:
del index, vector_store, storage_context, query_engine

vector_store = MilvusVectorStore(overwrite=False)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context)
query_engine = index.as_query_engine()
res = query_engine.query("What is the number?")
print("Res:", res)

Res: 
The number is ten.


In [9]:
res = query_engine.query("Who is the author?")
print("Res:", res)

Res: 
The author is Paul Graham.
